<a href="https://colab.research.google.com/github/ajsbsd/generated-images/blob/main/SDXL_no_nsfw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
#!pip install diffusers transformers accelerate bitsandbytes Pillow requests torch
"""
# Stable Diffusion Image Generation with NSFW Detection (Local)

This script provides a comprehensive solution for generating images using Stable Diffusion
on a local machine, incorporating a post-processing NSFW detection step.

Key Features:
- **Stable Diffusion Image Generation**: Utilizes `diffusers` to create images from prompts.
- **Configurable Parameters**: Easily adjust prompts, negative prompts, steps, and guidance scale.
- **Metadata Embedding**: Saves generated images as PNGs with embedded generation parameters.
- **NSFW Detection**: Includes a placeholder for a CLIP-based NSFW detector
  to filter out undesirable content after generation. Images flagged as NSFW are
  quarantined to a separate directory. The detection model is only loaded if a CUDA GPU is available.

**Important Note on NSFW Detection:**
The NSFW detection implemented here is **conceptual by default**. For a robust solution,
you'll need to uncomment and configure the actual CLIP model loading.
The placeholder `is_nsfw` function simulates detection based on a random chance
or keyword presence if the actual CLIP model is not loaded.
"""

# === Standard Library Imports ===
import os
from datetime import datetime
import random
import time

# === Third-Party Library Imports ===
import torch
from PIL import Image, PngImagePlugin
from diffusers import StableDiffusionPipeline, AutoencoderKL

# === NSFW Detection Libraries ===
# IMPORTANT: Uncomment these imports if you want to use a real CLIP-based NSFW detector.
# You will also need to install the 'transformers' library:
# pip install transformers
import torch.nn.functional as F # <-- UNCOMMENT THIS LINE
from transformers import CLIPProcessor, CLIPModel # <-- UNCOMMENT THIS LINE


# === Configuration ===
MODEL_ID = "runwayml/stable-diffusion-v1-5"
OUTPUT_DIR = "generated_images"
NSFW_QUARANTINE_DIR = "nsfw_quarantined_images" # Directory for flagged images
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(NSFW_QUARANTINE_DIR, exist_ok=True) # Create quarantine directory

# Set a reasonable number of images to generate
NUM_IMAGES_TO_GENERATE = 5

BASE_PROMPT = "Romanian Brown Bear"
NEGATIVE_PROMPT = (
    "(nsfw:1.5), (easynegative:1.3) (bad_prompt:1.3) badhandv4 bad-hands-5 (negative_hand-neg) "
    "(bad-picture-chill-75v), (worst quality:1.3), (low quality:1.3), (bad quality:1.3), "
    "(a shadow on skin:1.3), (a shaded skin:1.3), (a dark skin:1.3), (blush:1.3), "
    "(signature, watermark, username, letter, copyright name, copyright, chinese text, artist name, name tag, "
    "company name, name tag, text, error:1.5), (bad anatomy:1.5), (low quality hand:1.5), (worst quality hand:1.5)"
)

GENERATION_CONFIG = {
    "vae": "stabilityai/sd-vae-ft-mse",
    "sampler": "Euler a", # Note: Diffusers pipeline uses `scheduler` parameter for sampler
    "steps": 25,
    "guidance_scale": 7.0
}

# === Initialize Model ===
# Determine the device for model execution
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

# Determine the dtype based on the device
# Use float16 for CUDA (GPU) for performance, else use float32 for CPU compatibility
DTYPE = torch.float16 if DEVICE == "cuda" else torch.float32
print(f"Using dtype: {DTYPE}")

# Load the VAE model separately if specified
vae = None
if "vae" in GENERATION_CONFIG and GENERATION_CONFIG["vae"]:
    try:
        vae = AutoencoderKL.from_pretrained(GENERATION_CONFIG["vae"], torch_dtype=DTYPE).to(DEVICE)
        print(f"Loaded VAE: {GENERATION_CONFIG['vae']}")
    except Exception as e:
        print(f"Warning: Could not load VAE {GENERATION_CONFIG['vae']}. Error: {e}. Proceeding without it.")
        vae = None
## 05.24.2025 - ajsbsd.net python dev library

## 11.01.2023 - ajsbsd.net website with Hugging Face Chat inference

[Live at ajsbsd.net](https://ajsbsd.net)

## Tools

- [Debian/bookworm](https://debian.org)
- [Apache](https://httpd.apache.org)
- [NextJS 14.0.1 now running in Docker](https://nextjs.org)
- [TailwindCSS](https://tailwindcss.com/docs/customizing-colors)

## Goals

- Interact with HF API similar to [Vercel Hugging Face Guide](https://sdk.vercel.ai/docs/guides/providers/hugging-face)
- Learn how to make UI look tolerable without Radix-UI Theme
- Experiment with HF Inference since my server won't run OpenLLM/BentoML very well :)

## Docker

### 11.28.2023 - Dockerfile live at [ajsbsd.net](https://ajsbsd.net)

```bash
$ docker build -t ajsbsd.net-nextjs-docker .
$ docker run -p 127.0.0.1:3000:3000 ajsbsd.net-nextjs-docker

# Initialize the Stable Diffusion pipeline
# Pass DTYPE to torch_dtype for dynamic precision
pipe = StableDiffusionPipeline.from_pretrained(MODEL_ID, torch_dtype=DTYPE, vae=vae)
pipe.to(DEVICE)
# Enable attention slicing for lower VRAM usage, only if on CUDA
if DEVICE == "cuda":
    pipe.enable_attention_slicing()
    print("Attention slicing enabled for CUDA device.")

# === Initialize NSFW Detector (Conceptual/Optional Real) ===
# IMPORTANT: Uncomment the following lines if you want to use a real CLIP-based NSFW detector.
# Using 'laion/CLIP-ViT-B-32-laion2B-s34B-b79K' is a good starting point.
nsfw_detector_model = None
nsfw_detector_processor = None
try:
    if DEVICE == "cpu":
        print("NSFW Detector: Running on CPU, skipping full CLIP model load for performance.")
    else:
        print("Attempting to load CLIP-based NSFW detector...")
        nsfw_detector_model = CLIPModel.from_pretrained("laion/CLIP-ViT-B-32-laion2B-s34B-b79K").to(DEVICE) # <-- THIS LINE WILL NOW WORK
        nsfw_detector_processor = CLIPProcessor.from_pretrained("laion/CLIP-ViT-B-32-laion2B-s34B-b79K") # <-- THIS LINE WILL NOW WORK
        print("✅ CLIP-based NSFW Detector initialized.")
except ImportError:
    print("❌ 'transformers' library not found. Please install it for real NSFW detection.")
    print("   `pip install transformers`")
except Exception as e:
    print(f"❌ Could not initialize CLIP-based NSFW Detector. Error: {e}. NSFW detection will be conceptual.")

# Placeholder for conceptual NSFW detection if real one isn't loaded/fails
# This line is now effectively the fallback if the above 'try' block fails
# nsfw_detector_model = None
# nsfw_detector_processor = None
# print("NSFW detection will be conceptual unless CLIP model loading is uncommented and successful.")


# --- Functions ---

def add_metadata_and_save(image: Image.Image, filepath: str, prompt: str, negative_prompt: str, seed: int):
    """
    Embeds generation metadata into a PNG image and saves it to the specified filepath.
    """
    meta = PngImagePlugin.PngInfo()
    meta.add_text("Prompt", prompt)
    meta.add_text("NegativePrompt", negative_prompt)
    meta.add_text("Model", MODEL_ID)
    meta.add_text("VAE", GENERATION_CONFIG.get("vae", "N/A"))
    meta.add_text("Sampler", GENERATION_CONFIG.get("sampler", "N/A"))
    meta.add_text("Steps", str(GENERATION_CONFIG.get("steps", "N/A")))
    meta.add_text("Seed", str(seed))
    meta.add_text("Date", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

    image.save(filepath, "PNG", pnginfo=meta)
    print(f"Metadata added and image saved to: {filepath}")


def is_nsfw(image: Image.Image) -> tuple[bool, float]:
    """
    Detects NSFW content.
    If a real CLIP-based detector is loaded, it will be used.
    Otherwise, it falls back to a conceptual, random detection.
    """
    if nsfw_detector_model is not None and nsfw_detector_processor is not None:
        # --- START OF REAL CLIP-BASED NSFW DETECTION (if uncommented and loaded) ---
        # Ensure F (torch.nn.functional) is imported if using F.normalize
        # from transformers import CLIPProcessor, CLIPModel
        # import torch.nn.functional as F

        inputs = nsfw_detector_processor(images=image, return_tensors="pt").to(DEVICE)
        image_features = nsfw_detector_model.get_image_features(**inputs)

        safe_text_prompts = ["a photo of a person", "a general image", "a family-friendly photo"]
        unsafe_text_prompts = ["a photo of nudity", "a pornographic image", "an offensive image", "explicit content"]

        text_inputs_safe = nsfw_detector_processor(text=safe_text_prompts, return_tensors="pt", padding=True).to(DEVICE)
        text_features_safe = nsfw_detector_model.get_text_features(**text_inputs_safe)

        text_inputs_unsafe = nsfw_detector_processor(text=unsafe_text_prompts, return_tensors="pt", padding=True).to(DEVICE)
        text_features_unsafe = nsfw_detector_model.get_text_features(**text_inputs_unsafe)

        # Normalize embeddings for cosine similarity
        image_features = F.normalize(image_features, p=2, dim=-1)
        text_features_safe = F.normalize(text_features_safe, p=2, dim=-1)
        text_features_unsafe = F.normalize(text_features_unsafe, p=2, dim=-1)
## 05.24.2025 - ajsbsd.net python dev library

## 11.01.2023 - ajsbsd.net website with Hugging Face Chat inference

[Live at ajsbsd.net](https://ajsbsd.net)

## Tools

- [Debian/bookworm](https://debian.org)
- [Apache](https://httpd.apache.org)
- [NextJS 14.0.1 now running in Docker](https://nextjs.org)
- [TailwindCSS](https://tailwindcss.com/docs/customizing-colors)

## Goals

- Interact with HF API similar to [Vercel Hugging Face Guide](https://sdk.vercel.ai/docs/guides/providers/hugging-face)
- Learn how to make UI look tolerable without Radix-UI Theme
- Experiment with HF Inference since my server won't run OpenLLM/BentoML very well :)

## Docker

### 11.28.2023 - Dockerfile live at [ajsbsd.net](https://ajsbsd.net)

```bash
$ docker build -t ajsbsd.net-nextjs-docker .
$ docker run -p 127.0.0.1:3000:3000 ajsbsd.net-nextjs-docker

        safe_similarity = (image_features @ text_features_safe.T).mean().item()
        unsafe_similarity = (image_features @ text_features_unsafe.T).mean().item()

        # Simple classification based on which similarity is higher and a threshold
        nsfw_threshold = 0.25 # Adjust based on experimentation
        if unsafe_similarity > safe_similarity and unsafe_similarity > nsfw_threshold:
            print(f"  🚨 Detected as potentially NSFW (CLIP-based: Unsafe {unsafe_similarity:.2f} > Safe {safe_similarity:.2f}).")
            return True, unsafe_similarity
        else:
            return False, unsafe_similarity
        # --- END OF REAL CLIP-BASED NSFW DETECTION ---

    # --- START OF CONCEPTUAL/FALLBACK NSFW DETECTION LOGIC ---
    # This runs if the real CLIP model is not loaded or if the above real logic isn't fully implemented.
    if "nude" in BASE_PROMPT.lower() or random.random() < 0.15: # 15% chance to be flagged NSFW for demo
        print("  🚨 Detected as potentially NSFW (conceptual detection).")
        return True, random.uniform(0.6, 0.98) # Simulate high confidence
    else:
        return False, random.uniform(0.01, 0.3) # Simulate low confidence
    # --- END OF CONCEPTUAL/FALLBACK NSFW DETECTION LOGIC ---


def generate_and_process_images(num_images: int = 1):
    """
    Generates a specified number of images, performs NSFW detection,
    and saves them with metadata.
    """
    for i in range(num_images):
        variation = ", vibrant colors, neon lights" if i % 2 == 0 else ", soft pastel tones, morning light"
        current_prompt = BASE_PROMPT + variation
        seed = random.randint(10000000, 99999999)
        generator = torch.Generator(device=DEVICE).manual_seed(seed)

        print(f"\n--- Generating image {i + 1}/{num_images} with seed {seed} ---")
        print(f"Prompt: {current_prompt}")

        try:
            result = pipe(
                prompt=current_prompt,
                negative_prompt=NEGATIVE_PROMPT,
                num_inference_steps=GENERATION_CONFIG["steps"],
                guidance_scale=GENERATION_CONFIG["guidance_scale"],
                generator=generator,
            )
            image = result.images[0]
        except Exception as e:
            print(f"❌ Image generation failed for image {i+1}: {e}")
            continue

        # --- NSFW Detection Step ---
        is_flagged, confidence = is_nsfw(image)
        if is_flagged:
            print(f"!!! Image flagged as NSFW (confidence: {confidence:.2f}). Moving to quarantine. !!!")
            filename = os.path.join(NSFW_QUARANTINE_DIR, f"nsfw_image_{datetime.now().strftime('%Y%m%d_%H%M%S_%f')}_{i}.png")
            add_metadata_and_save(image, filename, current_prompt, NEGATIVE_PROMPT, seed)
            print(f"Quarantined: {filename}")
            continue

        print(f"Image passed NSFW check (confidence: {confidence:.2f}). Proceeding to save.")
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
        filename = os.path.join(OUTPUT_DIR, f"image_{timestamp}_{i}.png")

        add_metadata_and_save(image, filename, current_prompt, NEGATIVE_PROMPT, seed)


# === Execution ===
if __name__ == "__main__":
    print("\n--- Starting Stable Diffusion Image Generation ---")
    try:
        generate_and_process_images(num_images=NUM_IMAGES_TO_GENERATE)
    except Exception as e:
        print(f"An unexpected error occurred during the process: {e}")
    finally:
        # Clean up GPU memory
        del pipe
        if 'vae' in locals() and vae is not None:
            del vae
        # Uncomment these lines if you enable and load the real NSFW detector
        if 'nsfw_detector_model' in locals() and nsfw_detector_model is not None: # <-- UNCOMMENT THIS LINE
            del nsfw_detector_model # <-- UNCOMMENT THIS LINE
        if 'nsfw_detector_processor' in locals() and nsfw_detector_processor is not None: # <-- UNCOMMENT THIS LINE
            del nsfw_detector_processor # <-- UNCOMMENT THIS LINE
        torch.cuda.empty_cache()
        print("\n--- Process finished. GPU memory cleared. ---")
        print(f"Generated images saved to: {OUTPUT_DIR}")
        print(f"NSFW flagged images (if any) saved to: {NSFW_QUARANTINE_DIR}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loaded VAE: stabilityai/sd-vae-ft-mse


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

NameError: name 'CLIPModel' is not defined